In [ ]:
# Import necessary libraries
import os  # Library for operating system-related functions
import numpy as np  # Numerical computing library
import matplotlib.pyplot as plt  # Plotting library
import seaborn as sns  # Statistical data visualization library
import tensorflow as tf  # TensorFlow machine learning framework
import tensorflow_datasets as tfds  # TensorFlow Datasets library
import tensorflow_hub as hub  # TensorFlow Hub for pre-trained models
import tensorflow_addons as tfa  # TensorFlow Addons for additional functions
from tensorflow.keras.layers import Dense, Dropout  # Layers for building neural networks
from tensorflow.keras.callbacks import EarlyStopping  # Callback for early stopping during training

## Downloading Dataset from TensorFlow Datasets

In [ ]:
# importing all the images from meta data
all_data= tfds.load("eurosat", with_info=True) 

#Dividing dataset into train, validation and test
train_ds = tfds.load("eurosat", split="train[:60%]")
test_ds  = tfds.load("eurosat", split="train[60%:80%]")
valid_ds = tfds.load("eurosat", split="train[80%:]")

#Storing Class names
class_names = all_data[1].features["label"].names
num_classes = len(class_names)
num_examples = all_data[1].splits["train"].num_examples

In [ ]:
# Plotting a bar graph to visualize class distribution
fig, ax = plt.subplots(1, 1, figsize=(10,5))

# Counting the occurrences of each class label
labels, counts = np.unique(np.fromiter(all_data[0]["train"].map(lambda x: x["label"]), np.int32), return_counts=True)

# Rotate x-axis labels for better readability
plt.xticks(rotation=90)

# Set y and x axis labels
plt.ylabel('Count')
plt.xlabel('Classes')

# Create a bar plot using seaborn
sns.barplot(x=[class_names[l] for l in labels], y=counts, ax=ax, color=None)

# Add count annotations to the bars
for i, x in enumerate(labels):
    ax.text(x - 0.2, counts[i] + 5, counts[i])

# Set the title of the plot
ax.set_title("Number per class")

# Save the plot as an image
plt.savefig('sample_plot.png')

# Display the plot
plt.show()


## Data Preprocessing

In [ ]:
# Function to apply data augmentation to an image-label pair
def augment(image, label):
    # Randomly flip the image horizontally
    image = tf.image.random_flip_left_right(image)
    
    # Randomly adjust the brightness of the image
    image = tf.image.random_brightness(image, max_delta=0.1)
    
    # Randomly adjust the contrast of the image
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    
    return image, label


# Function to prepare a dataset for training
def prepare_for_training(dataset, cache=True, augment_data=True, batch_size=64, shuffle_buffer_size=1000):
    # Cache the dataset if specified
    if cache:
        if isinstance(cache, str):
            ds = dataset.cache(cache)
        else:
            ds = dataset.cache()

    # Map each data point to an image-label pair and one-hot encode the label
    ds = ds.map(lambda d: (d["image"], tf.one_hot(d["label"], num_classes)))

    # Apply data augmentation if specified
    if augment_data:
        ds = ds.map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Shuffle the dataset and repeat indefinitely
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    
    # Batch the dataset
    ds = ds.batch(batch_size)
    
    # Prefetch the data for better performance
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return ds


In [ ]:
# Define the batch size for training and validation
batch_size = 64

# Prepare the training dataset with data augmentation and specified batch size
train_ds = prepare_for_training(train_ds, augment_data=True, batch_size=batch_size)

# Prepare the validation dataset with the specified batch size (no augmentation)
valid_ds = prepare_for_training(valid_ds, batch_size=batch_size)

# Loop over a small portion of the validation dataset to print the shape of an example
for example in valid_ds.take(1):
    # Print the shape of the input image and its corresponding label
    print("Validation Example - Image Shape:", example[0].shape, "Label Shape:", example[1].shape)

# Loop over a small portion of the training dataset to print the shape of an example
for example in train_ds.take(1):
    # Print the shape of the input image and its corresponding label
    print("Training Example - Image Shape:", example[0].shape, "Label Shape:", example[1].shape)


In [ ]:
def show_batch(batch):
    # Create a grid of subplots to display the batch of images and labels
    plt.figure(figsize=(20, 20))
    
    # Loop through the batch and display each image and its corresponding label
    for n in range(min(32, batch_size)):
        ax = plt.subplot(batch_size//8, 8, n + 1)
        
        # Display the image
        plt.imshow(batch[0][n])
        
        # Set the title of the subplot as the corresponding label (class name)
        plt.title(class_names[tf.argmax(batch[1][n].numpy())])
        
        # Turn off axis labels
        plt.axis('off')
    
    # Adjust the spacing between images
    plt.subplots_adjust(wspace=0.15, hspace=0.10) 
    
    # Save the plot as an image
    plt.savefig("first_batch.png")
    
    # Display the plot
    plt.show()

# Call the show_batch function to display a batch of images from the training dataset
show_batch(next(iter(train_ds)))


In [ ]:
# Define the pre-trained model URL
model_handle = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/feature_vector/2"

# Create a KerasLayer using the pre-trained model for feature extraction
feature_extractor = hub.KerasLayer(model_handle, output_shape=[1280], trainable=True)

# Build the sequential model
model = tf.keras.Sequential([
    feature_extractor,
    Dropout(0.5),  # Adding dropout with a rate of 0.5 (50% of units dropped out)
    Dense(num_classes, activation="softmax")
])

# Build the model with a specified input shape
model.build([None, 64, 64, 3]) # input shape

# Compile the model
model.compile(
    loss="categorical_crossentropy", 
    #optimizer="Adam", 
    optimizer="RMSprop", 
    metrics=["accuracy", tfa.metrics.F1Score(num_classes)])


In [ ]:
#Model Summary
model.summary()

In [ ]:
# Define the name and path for saving the trained model
model_name = "satellite_image_classifier"
model_path = os.path.join("output", model_name + ".h5")

# Define callbacks for model training
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Calculate the number of training and validation steps based on data size and batch size
n_training_steps   = int(num_examples * 0.6) // batch_size
n_validation_steps = int(num_examples * 0.2) // batch_size

# Train the model using the training and validation datasets
history = model.fit(
    train_ds, 
    validation_data=valid_ds,
    steps_per_epoch=n_training_steps,
    validation_steps=n_validation_steps,
    verbose=1, epochs=50, 
callbacks=[model_checkpoint, early_stop]  # Adding early_stop callback here
)


In [ ]:
# Load the trained model weights from the specified model path
model.load_weights(model_path)

# Calculate the number of testing steps based on a portion of the training dataset
n_testing_steps = int(all_data[1].splits["train"].num_examples * 0.2)

# Get images and labels from the test dataset for evaluation
images = np.array([x["image"] for x in test_ds.take(n_testing_steps)])
print("images shape:", images.shape)
labels = np.array([x["label"] for x in test_ds.take(n_testing_steps)])
print("labels shape:", labels.shape)

# Make predictions using the loaded model
predictions = model.predict(images)
predictions = np.argmax(predictions, axis=1)  # Convert probabilities to class indices
print("predictions.shape:", predictions.shape)

# Calculate accuracy and F1 score for model evaluation
from sklearn.metrics import f1_score

accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(labels, predictions)
print("Accuracy:", accuracy.result().numpy())

f1_macro = f1_score(labels, predictions, average="macro")
print("F1 Score (Macro):", f1_macro)

In [ ]:
# Compute the confusion matrix between the true labels and the predicted labels
cm = tf.math.confusion_matrix(labels, predictions).numpy()

# Normalize the confusion matrix to show percentages along each column
cm = cm.astype('float') / cm.sum(axis=0)[:, np.newaxis]

# Create a figure and axis for the heatmap visualization
fig, ax = plt.subplots(figsize=(10, 10))

# Create a heatmap of the normalized confusion matrix with annotations
sns.heatmap(cm, annot=True, fmt='.2f', 
            xticklabels=[f"{c}" for c in class_names], 
            yticklabels=[f"{c}" for c in class_names],
            cmap="rocket_r"  # Choose the color map for better visibility
            )

# Set labels for the x-axis and y-axis
plt.ylabel('Actual')
plt.xlabel('Predicted')

# Save the heatmap visualization as an image
plt.savefig("confusion_matrix.png")


In [ ]:
# Define a function to visualize a grid of predicted samples
def show_predicted_samples():
    plt.figure(figsize=(14, 14))
    
    # Loop through a range of samples
    for n in range(64):
        ax = plt.subplot(8, 8, n + 1)
        
        # Display the image
        plt.imshow(images[n])
        
        # If the prediction matches the true label, set title in green
        if predictions[n] == labels[n]:
            ax.set_title(class_names[predictions[n]], color="green")
        # If the prediction doesn't match the true label, set title in red
        else:
            ax.set_title(f"{class_names[predictions[n]]}/T:{class_names[labels[n]]}", color="red")
        
        # Turn off axis labels
        plt.axis('off')
    
    # Save the plot as an image
    plt.savefig("predicted-sample-images.png")

# Call the show_predicted_samples function to display and save predicted sample images
show_predicted_samples()

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot training history metrics (accuracy and loss)
def plot_training_history(history):
    fig, axs = plt.subplots(2, 1, figsize=(7, 7))
    
    # Plot training & validation accuracy values
    axs[0].plot(history.history['accuracy'])
    axs[0].plot(history.history['val_accuracy'])
    axs[0].set_title('Model accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['Train', 'Validation'], loc='upper left')
    
    # Plot training & validation loss values
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set_title('Model loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['Train', 'Validation'], loc='upper left')
    
    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()

# Call the function to plot training history metrics
plot_training_history(history)
